In [245]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
from sklearn.model_selection import GridSearchCV

In [246]:
df =pd.read_csv("train-data.csv")
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [247]:
df.columns

Index(['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven',
       'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power',
       'Seats', 'New_Price', 'Price'],
      dtype='object')

In [248]:
df = df.drop(['Unnamed: 0'],axis =1)

In [249]:
df['Brand'] = df['Name'].apply(lambda x : x.split(' ')[0].lower())
df['Brand']

0              maruti
1             hyundai
2               honda
3              maruti
4                audi
5             hyundai
6              nissan
7              toyota
8          volkswagen
9                tata
10             maruti
11              honda
12             maruti
13               land
14               land
15         mitsubishi
16              honda
17             maruti
18            renault
19      mercedes-benz
20                bmw
21             maruti
22               audi
23            hyundai
24         volkswagen
25              honda
26             nissan
27             maruti
28              honda
29             toyota
            ...      
5989             audi
5990             tata
5991           maruti
5992            skoda
5993          hyundai
5994            honda
5995          renault
5996           jaguar
5997           maruti
5998       volkswagen
5999             tata
6000          hyundai
6001             audi
6002       volkswagen
6003      

In [250]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Brand
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75,maruti
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,hyundai
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,honda
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,maruti
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,audi


In [251]:
df = df.drop(['Name'],axis=1)

In [252]:
df.shape

(6019, 13)

In [253]:
df.describe()

,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468
std,3.269742,9.126884e+04,0.808840,11.187917
min,1998.000000,1.710000e+02,0.000000,0.440000
25%,2011.000000,3.400000e+04,5.000000,3.500000
50%,2014.000000,5.300000e+04,5.000000,5.640000
75%,2016.000000,7.300000e+04,5.000000,9.950000
max,2019.000000,6.500000e+06,10.000000,160.000000


In [254]:
df.isna().sum()

Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
Brand                   0
dtype: int64

In [255]:
df = df.dropna(axis =0,subset=['Mileage'])

In [256]:
df['Engine'].value_counts()

1197 CC    606
1248 CC    512
1498 CC    304
998 CC     259
2179 CC    240
1497 CC    229
1198 CC    227
1968 CC    216
1995 CC    183
1461 CC    152
2143 CC    149
1582 CC    145
1199 CC    143
1598 CC    141
1396 CC    139
796 CC     129
2494 CC    121
1086 CC    108
1591 CC     94
2993 CC     90
1399 CC     88
2982 CC     86
1798 CC     84
2987 CC     67
2967 CC     61
814 CC      59
1120 CC     54
1196 CC     50
1373 CC     47
1493 CC     47
          ... 
3597 CC      2
1242 CC      2
2997 CC      2
1781 CC      2
970 CC       1
2773 CC      1
2487 CC      1
2112 CC      1
4951 CC      1
5204 CC      1
2092 CC      1
2495 CC      1
2706 CC      1
1468 CC      1
1489 CC      1
2995 CC      1
2720 CC      1
2925 CC      1
2149 CC      1
1422 CC      1
1948 CC      1
1978 CC      1
1797 CC      1
3197 CC      1
5998 CC      1
2349 CC      1
2999 CC      1
2147 CC      1
2694 CC      1
3200 CC      1
Name: Engine, Length: 145, dtype: int64

In [257]:
df['Location'] = df['Location'].apply(lambda x: x.lower())

In [258]:
cat_col = ['Location', 'Fuel_Type', 'Transmission','Owner_Type','Brand']

df_dummies = pd.get_dummies(data=df.copy(), columns=cat_col, drop_first=True)

In [259]:
df_dummies.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,New_Price,Price,Location_bangalore,Location_chennai,...,Brand_mitsubishi,Brand_nissan,Brand_porsche,Brand_renault,Brand_skoda,Brand_smart,Brand_tata,Brand_toyota,Brand_volkswagen,Brand_volvo
0,2010,72000,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015,41000,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2011,46000,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2012,87000,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2013,40670,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,0,0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
df_dummies['Seats'].fillna(df_dummies['Seats'].mean(),inplace=True)
df_dummies.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,New_Price,Price,Location_bangalore,Location_chennai,...,Brand_mitsubishi,Brand_nissan,Brand_porsche,Brand_renault,Brand_skoda,Brand_smart,Brand_tata,Brand_toyota,Brand_volkswagen,Brand_volvo
0,2010,72000,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015,41000,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2011,46000,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2012,87000,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2013,40670,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,0,0,...,0,0,0,0,0,0,0,0,0,0


In [261]:
df_dummies.isna().sum()

Year                            0
Kilometers_Driven               0
Mileage                         0
Engine                         36
Power                          36
Seats                           0
New_Price                    5194
Price                           0
Location_bangalore              0
Location_chennai                0
Location_coimbatore             0
Location_delhi                  0
Location_hyderabad              0
Location_jaipur                 0
Location_kochi                  0
Location_kolkata                0
Location_mumbai                 0
Location_pune                   0
Fuel_Type_Diesel                0
Fuel_Type_LPG                   0
Fuel_Type_Petrol                0
Transmission_Manual             0
Owner_Type_Fourth & Above       0
Owner_Type_Second               0
Owner_Type_Third                0
Brand_audi                      0
Brand_bentley                   0
Brand_bmw                       0
Brand_chevrolet                 0
Brand_datsun  

In [279]:
df_dummies['Engine']=df_dummies['Engine'].str.replace('CC', '')

In [177]:
df_dummies['Engine']=df_dummies['Engine'].str.replace('NaN', '1700')

In [280]:
df_dummies['Engine'] = df_dummies['Engine'].astype(float)

In [269]:
import re

df_dummies['Mileage'] = df_dummies['Mileage'].apply(lambda x: re.sub(r'(\d+\.\d+)\s(kmpl|km\/kg)', r'\1', str(x)))

In [266]:
#re.findall('\d+', str(df_dummies['Mileage'][0]))

['26', '6']

In [267]:
#re.findall("[+-]?\d+\.\d+", df_dummies['Mileage'][0])

'26.6 km/kg'

In [285]:
df_dummies.Engine = df_dummies.Engine.fillna(df_dummies.Engine.mean())

In [271]:
df_dummies['Mileage'] = df_dummies['Mileage'].astype(float)
df_dummies['Mileage'][0]

26.6

In [286]:
df_dummies = df_dummies.dropna(axis =0,subset=['Engine'])
df_dummies

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,New_Price,Price,Location_bangalore,Location_chennai,...,Brand_mitsubishi,Brand_nissan,Brand_porsche,Brand_renault,Brand_skoda,Brand_smart,Brand_tata,Brand_toyota,Brand_volkswagen,Brand_volvo
0,2010,72000,26.60,998.0,58.16 bhp,5.0,NaN,1.75,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015,41000,19.67,1582.0,126.2 bhp,5.0,NaN,12.50,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2011,46000,18.20,1199.0,88.7 bhp,5.0,8.61 Lakh,4.50,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2012,87000,20.77,1248.0,88.76 bhp,7.0,NaN,6.00,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2013,40670,15.20,1968.0,140.8 bhp,5.0,NaN,17.74,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2012,75000,21.10,814.0,55.2 bhp,5.0,NaN,2.35,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2013,86999,23.08,1461.0,63.1 bhp,5.0,NaN,3.50,0,0,...,0,1,0,0,0,0,0,0,0,0
7,2016,36000,11.36,2755.0,171.5 bhp,8.0,21 Lakh,17.50,0,0,...,0,0,0,0,0,0,0,1,0,0
8,2013,64430,20.54,1598.0,103.6 bhp,5.0,NaN,5.20,0,0,...,0,0,0,0,0,0,0,0,1,0
9,2012,65932,22.30,1248.0,74 bhp,5.0,NaN,1.95,0,1,...,0,0,0,0,0,0,1,0,0,0


In [290]:
df_dummies["Power"] = df_dummies["Power"].apply(lambda x : re.findall("[+-]?\d+\.\d+", x))

In [294]:
df_dummies["Power"] = df_dummies["Power"].apply(lambda x : x[0] if len(x)>0 else 0)

In [309]:
df_dummies["Power"] = df_dummies["Power"].astype(float)

In [310]:
X = df_dummies.drop(['New_Price','Price'],axis=1)
y = df_dummies['Price']

In [311]:
print(X.shape)
print(y.shape)

(5981, 52)
(5981,)


In [312]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [316]:
y_train

90       5.00
2683     4.99
3337     6.70
3687     8.27
482      4.24
3449     4.65
157      2.50
5344     5.50
3018     4.00
5791    38.76
4405     3.75
2604    35.00
3592     4.94
1048     4.90
4630    13.90
4172     3.90
5547    11.94
2787    34.54
1699     3.00
2027     1.50
3547     3.25
4035     2.00
4849     5.70
917      5.25
1266     2.40
773      1.50
3        6.00
232      3.60
1496    58.73
1426     5.95
        ...  
1035    11.00
1114     6.85
1896     7.27
3574    19.50
1535     6.99
757      3.25
3069     2.40
2644     5.90
564      5.65
4797     4.50
3586     2.60
254      1.75
2529    10.00
2982    14.76
4482     1.60
5412    13.85
5434     8.20
1206     7.75
3484    12.00
2816     7.31
4253     3.15
144     13.49
5092     5.30
2915     5.85
2782     5.85
909      6.46
5229    12.50
4005     6.45
237      3.87
5194     5.90
Name: Price, Length: 4485, dtype: float64

In [299]:
from sklearn.svm import SVR
model = SVR()
model.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [300]:
y_predict = model.predict(X_test)

In [301]:
r2_score(y_test, y_predict)

-0.0956394151665021

In [302]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr.fit(X_train,y_train)
y_predict_rf = regr.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [303]:
r2_score(y_test, y_predict_rf)

0.8765604761082493

In [304]:
import xgboost

In [313]:
xgb = xgboost.XGBRegressor()
xgb.fit(X_train,y_train)
y_predict_xgb = xgb.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:01:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [314]:
r2_score(y_test, y_predict_xgb)

0.8746130088279491